# Factored Representations & STRIPS

You're building a robot that needs to rearrange blocks. How hard can it be?

## The Problem

Let's say you have 10 blocks and a table. Each block can be:
- On the table
- On top of another block
- Clear (nothing on top)
- Blocked (something on top)

Using our classical search approach (BFS, A*), we'd enumerate every possible state:

```
State 1: A on table, B on table, C on A, D on B...
State 2: A on table, B on table, C on B, D on A...
State 3: A on B, B on table, C on D, D on table...
...
```

**How many states?** With 10 blocks, there are over 1 billion possible configurations.

A 6-joint robot arm? 360^6 = **2.2 billion states**.

Classical search breaks. You can't store a billion states in memory. You can't compute heuristics for each one. You can't even enumerate them.

## The Insight

Here's what's wasteful: Most things DON'T CHANGE when you move a block.

When you pick up Block A and move it to Block B, you don't need to re-specify:
- Where Block C is
- Where Block D is  
- Where Block E is
- ...

You're repeating 90% of the state description just to change 10%.

**What if we only describe what's TRUE?**

```
Current state:
  On(A, Table)
  On(B, Table)
  Clear(A)
  Clear(B)
```

That's it. Four facts. Not a billion enumerated states. Everything not listed is assumed false.

This is a **factored representation** - describing states through sets of predicates instead of enumeration.

And it changes everything.

In [6]:
from collections import defaultdict
from typing import Set, List, Tuple, Dict
from copy import deepcopy

print("Libraries loaded")

Libraries loaded


## The Next Problem: What Changes When?

Great, we describe states compactly. But how do actions work?

In classical search, you had a successor function:
```python
def successors(state):
    return [all_possible_next_states]
```

But with factored states, how do you specify what an action does without enumerating every possible before/after state pair?

## STRIPS: Describing Change

STRIPS (Stanford Research Institute Problem Solver) solves this by describing what CHANGES, not complete states.

Every action has three parts:

1. **Preconditions** - what must be true to execute  
2. **Add list** - what becomes true after
3. **Delete list** - what becomes false after

### Moving a Block

```
Action: Move(A, Table, B)

Preconditions:
  On(A, Table)    ← A must be on table
  Clear(A)         ← Nothing on A
  Clear(B)         ← Nothing on B

Add list:
  On(A, B)         ← A is now on B
  Clear(Table)     ← Table spot is now free

Delete list:
  On(A, Table)     ← A no longer on table
  Clear(B)         ← B no longer clear
```

Look at what's missing: Block C, Block D, everything else. They persist automatically.

**The STRIPS assumption:** Everything not in add/delete list stays unchanged.

This solves the **frame problem** - you don't need axioms specifying that unaffected facts persist.

In [7]:
class STRIPSAction:
    def __init__(self, name: str, preconditions: Set[str], 
                 add_effects: Set[str], delete_effects: Set[str]):
        self.name = name
        self.preconditions = preconditions
        self.add_effects = add_effects
        self.delete_effects = delete_effects
    
    def is_applicable(self, state: Set[str]) -> bool:
        """Can this action execute in the given state?"""
        return self.preconditions.issubset(state)
    
    def apply(self, state: Set[str]) -> Set[str]:
        """Execute action and return new state"""
        if not self.is_applicable(state):
            raise ValueError(f"Cannot apply {self.name} - preconditions not met")
        
        new_state = state.copy()
        new_state -= self.delete_effects  # Remove what becomes false
        new_state |= self.add_effects      # Add what becomes true
        return new_state
    
    def __repr__(self):
        return self.name

# Example: Classic blocks world
initial_state = {
    'On(A, Table)',
    'On(B, Table)', 
    'Clear(A)',
    'Clear(B)'
}

move_A_onto_B = STRIPSAction(
    name='Move(A, Table, B)',
    preconditions={'On(A, Table)', 'Clear(A)', 'Clear(B)'},
    add_effects={'On(A, B)', 'Clear(Table)'},
    delete_effects={'On(A, Table)', 'Clear(B)'}
)

print("Initial state:", sorted(initial_state))
new_state = move_A_onto_B.apply(initial_state)
print("After Move(A, Table, B):", sorted(new_state))
print("\nNotice: On(B, Table) unchanged - frame axiom handled automatically")

Initial state: ['Clear(A)', 'Clear(B)', 'On(A, Table)', 'On(B, Table)']
After Move(A, Table, B): ['Clear(A)', 'Clear(Table)', 'On(A, B)', 'On(B, Table)']

Notice: On(B, Table) unchanged - frame axiom handled automatically


## Why STRIPS Works

Look at what just happened:

**State transition in explicit enumeration:**
```
State_1 → State_2
(all 4 facts listed) → (all 4 facts listed again)
```

**State transition in STRIPS:**
```
Remove: On(A, Table), Clear(B)
Add: On(A, B), Clear(Table)
```

STRIPS focuses on **what changed**. The fact that On(B, Table) persists is handled implicitly.

**Key insight:** Most facts don't change during any single action. Why waste space repeating them?

### The Frame Problem

In logical planning, you'd need axioms like:
```
On(Block_C, Block_D) ∧ Move(Block_A, X, Y) → On(Block_C, Block_D)
```

For EVERY fact and EVERY action that doesn't affect it. With 100 facts and 20 actions, that's 2000 axioms.

**STRIPS eliminates this.** The default is persistence. Only changes need specification.

## The Reusability Problem

We've solved compactness and change description. But there's one more issue.

You write a blocks world planner. Then you need a logistics planner. Then a robot navigation planner. Each time, you're rewriting domain knowledge in code.

**What if domains were data, not code?**

## PDDL: Planning as Configuration

PDDL (Planning Domain Definition Language) separates what you know from what you're solving.

**Two files:**

1. **Domain file** - types, predicates, actions (write once)
2. **Problem file** - objects, initial state, goal (many instances)

Think of it like a game engine. The domain is the physics engine and rules. The problem is the specific level you're playing.

Write "blocks world" once. Solve 1000 different block arrangements without changing the domain.

In [8]:
class PDDLDomain:
    def __init__(self, name: str):
        self.name = name
        self.predicates = set()
        self.actions = {}
    
    def add_action(self, action: STRIPSAction):
        self.actions[action.name] = action

class PDDLProblem:
    def __init__(self, domain: PDDLDomain, objects: Set[str],
                 initial: Set[str], goal: Set[str]):
        self.domain = domain
        self.objects = objects
        self.initial = initial
        self.goal = goal

# Define domain (reusable)
blocks_domain = PDDLDomain('blocksworld')
blocks_domain.add_action(move_A_onto_B)

# Define problem instance 1
problem1 = PDDLProblem(
    domain=blocks_domain,
    objects={'A', 'B', 'Table'},
    initial={'On(A, Table)', 'On(B, Table)', 'Clear(A)', 'Clear(B)'},
    goal={'On(A, B)'}
)

# Define problem instance 2 (same domain, different setup)
problem2 = PDDLProblem(
    domain=blocks_domain,
    objects={'A', 'B', 'C', 'Table'},
    initial={'On(A, B)', 'On(B, Table)', 'On(C, Table)', 'Clear(A)', 'Clear(C)'},
    goal={'On(C, A)'}
)

print(f"Domain '{blocks_domain.name}' with {len(blocks_domain.actions)} actions")
print(f"Problem 1: {len(problem1.objects)} objects, goal: {problem1.goal}")
print(f"Problem 2: {len(problem2.objects)} objects, goal: {problem2.goal}")
print("\nSame domain, different problems. That's the power of separation.")

Domain 'blocksworld' with 1 actions
Problem 1: 3 objects, goal: {'On(A, B)'}
Problem 2: 4 objects, goal: {'On(C, A)'}

Same domain, different problems. That's the power of separation.


## But We Still Need Search

We can describe states compactly. We can specify actions generically. We can separate domains from problems.

But we still need to SEARCH. And search needs heuristics.

In classical search (like A*), you wrote custom heuristics for each domain:
- Manhattan distance for grids
- Graph distance for navigation  
- Custom logic for each problem

**With STRIPS, heuristics come for free:** The structure of actions lets you compute them automatically.

Same heuristic works for blocks world, logistics, robotics, scheduling - anything.

## Delete Relaxation: The Universal Heuristic

**The idea:** What if nothing could be undone?

```
Original action: Move(A, Table, B)
  Add: On(A, B)
  Delete: On(A, Table)  ← IGNORE THIS

Relaxed action:
  Add: On(A, B)
  Delete: nothing
```

In the relaxed world, A can be on both Table AND B. Impossible in reality, but easier to solve.

**Why this works:** If the easier problem takes N steps, the real problem takes ≥ N steps. Perfect admissible heuristic.

And it's computed purely from action structure - no domain knowledge needed.

In [9]:
def relaxed_action(action: STRIPSAction) -> STRIPSAction:
    """Create delete-relaxed version of action"""
    return STRIPSAction(
        name=f"{action.name}_relaxed",
        preconditions=action.preconditions,
        add_effects=action.add_effects,
        delete_effects=set()  # No deletes in relaxed problem
    )

def delete_relaxation_heuristic(state: Set[str], goal: Set[str], 
                                 actions: List[STRIPSAction]) -> int:
    """Count actions needed in relaxed problem"""
    relaxed_actions = [relaxed_action(a) for a in actions]
    
    current = state.copy()
    steps = 0
    
    while not goal.issubset(current):
        # Find action that adds something we need
        made_progress = False
        for action in relaxed_actions:
            if action.is_applicable(current):
                new_facts = action.add_effects - current
                if new_facts & goal:  # Adds something toward goal
                    current = action.apply(current)
                    steps += 1
                    made_progress = True
                    break
        
        if not made_progress:
            return float('inf')  # No solution
        
        if steps > 100:  # Prevent infinite loops
            return float('inf')
    
    return steps

state = {'On(A, Table)', 'Clear(A)'}
goal = {'On(A, B)'}
actions = [move_A_onto_B]

h = delete_relaxation_heuristic(state, goal, actions)
print(f"Delete relaxation estimate: {h} steps")
print("\nThis underestimates because relaxed problem ignores conflicts.")
print("But it's computed automatically from the action structure.")

Delete relaxation estimate: inf steps

This underestimates because relaxed problem ignores conflicts.
But it's computed automatically from the action structure.


### 2. Planning Graph Heuristic

**Idea:** Build a graph showing what facts become reachable at each time step.

```
Level 0 (initial state):
  On(A, Table), Clear(A), Clear(B)

Level 1 (after 1 action):
  On(A, B), Clear(Table)  [from Move(A, Table, B)]
  Plus everything from Level 0 that persists

Level 2 (after 2 actions):
  New facts reachable...
```

The level at which all goal facts first appear together gives a heuristic estimate.

### 3. Landmark Heuristic

**Idea:** Identify facts that MUST be true at some point in any solution.

```
Goal: On(A, B)
Landmark: Clear(A) must be true before we can move A
```

Count unachieved landmarks as heuristic estimate.

All three heuristics are computed from STRIPS structure alone. They work for logistics, blocks world, robotics, scheduling - any STRIPS domain.

## STRIPS Planner

Complete forward-search planner with domain-independent heuristics:

In [10]:
import heapq

def strips_planner(problem: PDDLProblem) -> List[STRIPSAction]:
    """A* search with delete-relaxation heuristic"""
    initial = frozenset(problem.initial)
    goal = problem.goal
    actions = list(problem.domain.actions.values())
    
    # Priority queue: (f_score, g_score, state, path)
    open_set = [(0, 0, initial, [])]
    visited = set()
    
    while open_set:
        f, g, state, path = heapq.heappop(open_set)
        
        if state in visited:
            continue
        visited.add(state)
        
        if goal.issubset(state):
            return path
        
        for action in actions:
            if action.is_applicable(set(state)):
                new_state = frozenset(action.apply(set(state)))
                new_path = path + [action]
                new_g = g + 1
                h = delete_relaxation_heuristic(set(new_state), goal, actions)
                new_f = new_g + h
                
                heapq.heappush(open_set, (new_f, new_g, new_state, new_path))
    
    return None

plan = strips_planner(problem1)
if plan:
    print("Plan found:")
    for i, action in enumerate(plan, 1):
        print(f"  {i}. {action}")
else:
    print("No plan found")

print("\nNotice: Same planner works for ANY PDDL domain.")
print("The heuristic is computed from action structure automatically.")

Plan found:
  1. Move(A, Table, B)

Notice: Same planner works for ANY PDDL domain.
The heuristic is computed from action structure automatically.


## What We Built

**Problem 1:** Billion-state spaces → **Solution:** Factored representations (describe what's true)

**Problem 2:** How to specify change? → **Solution:** STRIPS actions (describe what changes)

**Problem 3:** Rewriting domains as code → **Solution:** PDDL (domains as data)

**Problem 4:** Custom heuristics per domain → **Solution:** Delete relaxation (structure-based)

Each piece builds on the last.

**Classical search:**
```python
def solve_blocks_world():
    # Custom code for blocks
    states = enumerate_all_block_configs()
    heuristic = blocks_specific_logic()
    search(states, heuristic)

def solve_logistics():
    # Different custom code
    states = enumerate_all_truck_routes()
    heuristic = logistics_specific_logic()
    search(states, heuristic)
```

**STRIPS/PDDL:**
```python
def solve_anything(domain_file, problem_file):
    # Same code, different data
    problem = parse_pddl(domain_file, problem_file)
    heuristic = delete_relaxation()  # Works for all
    search(problem, heuristic)
```

Modern planners (Fast Downward, FastForward) solve problems with 10^100 states in seconds. Same planner, different PDDL files.

**The insight:** Representation matters more than algorithms. Structure enables generality.